In [101]:
import math


class Derivative:
    def __init__(self, f, h=0.0001):
        self.f = f
        self.h = float(h)

    def __call__(self, x):
        f, h = self.f, self.h
        return (f(x+h) - f(x-h))/(2*h)
class Derivative_1:
    def __init__(self, f, h=0.0001):
        self.f = f
        self.h = float(h)

    def __call__(self, x,y):
        f, h = self.f, self.h
        return (f(x+h,y+h) - f(x-h,y-h))/(2*h)



class Derivative2:
    def __init__(self, f, h=0.0001):
        self.f = f
        self.h = float(h)

    def __call__(self, x):
        f, h = self.f, self.h
        return (f(x+h) - 2 * f(x) + f(x-h))/h**2


def C(n, k):
    return math.factorial(n)/(math.factorial(n-k)*math.factorial(k))

def b(value):
    res = [0]
    summ = 0
    for i in range(1, value):
        summ += i
    for i in range(1, value):
        res.append(i/summ)
    return res


class Solver:

    def __init__(self, a, infin=3, fun = b):
        assert 0 < a < 1
        self.a = a
        self.inf = infin
        self.b = fun(self.inf)
        self.inA = 1 - a
        self.inB = self.calc_inB()
        self.pk = {}
        self.pik = {}
        self.gk = {}
        self.vk = {}
        assert self.calc_beta(self.inA)>self.inA/(1+self.inA)

    def calc_inB(self):
        sum = 0
        for i in range(self.inf):
            sum+=i*self.b[i]
        return sum

    def calc_Bi(self, i):
        sum = 0
        for j in range(i,self.inf):
            sum += self.b[j]
        return sum

    def calc_Bz(self, z):
        sum = 0
        for i in range(0,self.inf):
            sum += pow(z,i)*self.calc_Bi(i)
        return sum

    
    def calc_beta(self, z):
        sum = 0
        for i in range(self.inf):
            sum += z**i*self.b[i]
        return sum

    def calc_pk(self, k):
        if self.pk.get(k):
            return self.pk[k]
        else:
            if k==0:
                self.pk[k] = 1 - (self.inA*(1-self.calc_beta(self.inA))/self.calc_beta(self.inA))
            elif k==1:
                self.pk[k] = (1/self.calc_beta(self.inA)*(1-self.calc_beta(self.inA)))*self.calc_pk(0)
            else:
                self.pk[k] = (1/self.calc_beta(self.inA))*(1-self.calc_beta(self.inA))*(self.calc_pk(k-1)-self.calc_pik(1, k-1))
            return self.pk[k]

    def calc_pik(self, i, k):
        assert k>=1
        if self.pik.get((i, k)):
            return self.pik[(i, k)]
        else:
            if i==1 and k==1:
                self.pik[(i, k)] = self.a/self.inA*self.calc_pk(0)
            elif i==1:
                self.pik[(i, k)] = self.a/self.inA*(self.calc_pk(k-1) - self.calc_pik(1, k-1))
            elif k==1:
                sum = 0
                for j in range(i, self.inf):
                    sum+=self.inA**(j-i)*self.b[j]
                self.pik[(i, k)] = self.a*(self.calc_pk(0)+self.calc_pk(1))*sum
            else:
                sum = 0
                for j in range(i, self.inf):
                    sum += self.inA**(j-i)*self.b[j]
                self.pik[(i, k)] = self.a*(self.calc_pk(k-1)-self.calc_pik(1, k-1))*sum+self.a*self.calc_pk(k)*sum
            return self.pik[(i, k)]

    def calc_gk(self, k):
        if self.gk.get(k):
            return self.gk[k]
        else:
            if k==0:
                self.gk[k] = 0
            else:
                sum1, sum2 = 0, 0
                for i in range(1, k+1):
                    sum1 += self.inA**(i-1)*self.a*self.b[i]*self.calc_gk(k-i)
                    sum3 = 0
                    for j in range(k-i+1):
                        sum3 += self.calc_gk(j)*self.calc_gk(k-i-j)
                    sum2 += self.inA**(i-1)*self.a*self.calc_Bi(i+1)*sum3
                self.gk[k] = self.inA**k*self.b[k] + sum1 + sum2
            return self.gk[k]

    def calc_vk(self, k):
        if self.vk.get(k):
            return self.vk[k]
        else:
            if k==0:
                self.vk[k] = 0
            else:
                sum1 = 0
                for i in range(1,k+1):
                    sum2 = 0
                    for j in range(k-i+1):
                        sum2 += self.calc_gk(j)*self.calc_vk(k-i-j)
                    sum1 += self.inA**(i-1)*self.a*self.calc_Bi(i+1)*sum2
                self.vk[k] = self.inA**(k-1)*self.b[k]+sum1
            return self.vk[k]

    def calc_vkm(self, k, m):
        if k==0:
            return 0
        elif 1<=k<=m-1:
            sum1 = 0
            for i in range(1,k):
                sum2 = 0
                for j in range(0,k-i+1):
                    sum2 += self.calc_gk(j)*self.calc_vk(k-i-j)
                sum1 += self.inA**(i-1)*self.a*sum2
            return sum1
        else:
            sum1 = 0
            for i in range(1, m-1):
                sum2 = 0
                for j in range(0,k-i+1):
                    sum2 += self.calc_gk(j)*self.calc_vk(k-i-j)
                sum1 += self.inA**(i-1)*self.a*sum2
            return self.inA**(m-1) + sum1

    def calc_N(self):
        sum1 = 0
        for i in range(1, self.inf):
            sum2 = 0
            for k in range(1, self.inf):
                sum2 += k*self.calc_pik(i, k)
            sum1 += sum2
        return sum1
    def calc_ga(self,z):
        return(((1-self.inA*z)*(self.inA-self.a*self.calc_Bz(self.inA*z))-pow(((1-self.inA*z)*(1-self.inA*z)*((self.inA-self.a*self.calc_Bz(self.inA*z))*(self.inA-self.a*self.calc_Bz(self.inA*z))-4*self.a*(self.inA*z-self.calc_Bz(self.inA*z))*self.inA*(1-self.inA*(1-self.a*z)*self.calc_Bz(self.inA*z)))),(1/2)))/(2*self.a*(self.inA*z-self.calc_Bz(self.inA*z))))
    def calc_gamma1(self,z):
        der=Derivative((self.calc_ga))
        return(der(z))
    def calc_gamma2(self,z):
        der=Derivative((self.calc_ga))
        sum1=0
        for k in range(0,self.inf):
            sum1+=k*self.calc_gamma1(k)
        return(sum1)
    def calc_v1(self,z):
        der=Derivative((self.vk))
        return(der(1))
    def calc_vz(self,z):
        return((1-self.inA*z*self.calc_Bz(self.inA*z))/(self.inA*(1-self.inA*z)-self.a*(self.inA*z-self.calc_Bz(self.inA*z))*self.calc_ga(z)))
    def calc_v2(self):
        sum1=0
        for i in range(0,self.inf):
            sum1+=i*self.calc_vk(i)
        return(sum1)
    def calc_v(self,z,m):
        return((self.inA*z)**m/self.inA*self.a*z/self.inA*((1-self.inA*z)**(m-1)*self.calc_ga(z)*self.calc_vz(z)/(1-self.inA*z)))
    def calc_vm1(self,m):
        der=Derivative_1(self.calc_v)
        return(der(1,m))
    def calc_vm2(self,m):
        sum1=0
        for i in range(0,self.inf):
            sum1+=i*self.calc_vkm(i,m)
        return(sum1)
    
    
        

In [102]:
import PyGnuplot as gp


class Graphics:

    def _draw(self,
             points, # ([x1,y1],filename,functionname), ...
             xl='Значения a',
             yl='Значения функции',
             title='заголовок',
             yrange='[0:5]',
             xrange='[-1:1]',
             out_file='file.pdf'):
        gp.c('set xlabel "' + xl + '"')
        gp.c('set ylabel "' + yl + '"')
        gp.c('set title "' + title + '"')
        gp.c('set yrange ' + yrange)
        gp.c('set xrange ' + xrange)
        plotstr = 'plot '
        for q in points:
            gp.s([q[0][0], q[0][1]], filename=q[1])
            plotstr += '"' + q[1] + '" u 1:2 w l title "' + q[2] + '", '
        plotstr = plotstr.strip(', ')
        gp.c(plotstr)

    def draw_N1(self, a, j, da=0.001):
        a+=da
        s1 = Solver(a, j)
        s2 = Solver(a, j+1)
        x1 = []
        y1 = []
        x2 = []
        y2 = []
        while a < s1.inB**(-1):
            x1.append(a)
            y1.append(s1.calc_N1())
            a += da
            s1.reload(a, j)
        a = 0
        while a < s2.inB**(-1):
            x2.append(a)
            y2.append(s2.calc_N1())
            a += da
            s2.reload(a, j+1)




        points = []
        points.append(((x1, y1), 'tmp.dat', 'N1(a)'))
        points.append(((x2, y2), 'tmp2.dat', 'N1(2)(a)'))
        self._draw(points=points,
                   title="График зависимости N1(a) и N1(2)(a)",
                   )

In [103]:
# s = Solver(0.2, 4)
# print(s.calc_N1())
# print(s.calc_N2())
# print(s.calc_N3())
# print(s.calc_D())



g = Graphics()

g.draw_N1(0, 4)

AttributeError: 'Solver' object has no attribute 'calc_N1'

In [104]:
s = Solver(0.2, 4)

In [105]:
s.calc_vm1(2)

(-0.944159964165886-1.4007151234651793j)